# Unhappy Couple Flight

Source: https://artofproblemsolving.com/wiki/index.php/2024_AMC_8_Problems/Problem_25


## Problem

A small airplane has $4$ rows of seats with $3$ seats in each row. Seven passengers have boarded the plane and are distributed randomly among the seats. A married couple is next to board. 

**Q1:**
What is the probability there will be 2 adjacent seats in the same row for the couple?

**Q2:**
Suppose the couple just had a fight and one particular partner (mention no names) does not want to sit beside the other (and be scolded for the entire flight, hypothetically speaking). So they (see! gender neutral pronouns are useful!) pay for a few friends (who will be allocated random seats) to also take the flight, in the hope that the couple will be forced to sit apart. <br />
If they want this probability to be smaller than 25% (it was a bad fight) what is the fewest number of friends that will get a free flight? 

---

## Solution

### Setup

In [132]:
import numpy as np
rng = np.random.default_rng()

### Development Notes

* Step 1 - code snippets
  * Could treat the seats as an array from 0 to 11, but will need to put them into rows to represent the plane. Will use `.reshape` for this. 
    * Only need to remember if a seat is occupied or not - so use a `bool` array. **Q:** Does it make a difference if we use `True` for occupied or `False` for occupied?  
  * For any given row, the couple sit together if there are at least two seats free and the middle seat is free.
    * This is same as saying that the couple cannot sit together if there at least two occupied seats or the middle seat is occupied. 
* Step 2 - Implement functions
  * `simulate`, and `run_trials` as before.
  * Could have separate versions of the functions for each question, or have one version (since Q1 is a special case of Q2 (num friends = 0)).
* Step 3 - Validate functions
  * Test for probability convergence.
  * Does probability behave as expected for different numbers of friends? (increasing/decreasing as number of friends increases - think BEFORE running experiment so you are predicting and testing predictions.) 


In [148]:
# "create" seats - True is empty / False otherwise 
# Using True for free makes increasing plane size easier

seats = np.ones(12, dtype=bool)
seats

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

In [149]:
passengers = rng.choice(12, size=7, replace=False)
sorted(passengers)

[4, 5, 6, 7, 8, 10, 11]

In [150]:
seats[passengers] = False
seats

array([ True,  True,  True,  True, False, False, False, False, False,
        True, False, False])

In [151]:
plane = seats.reshape((4,3))
plane

array([[ True,  True,  True],
       [ True, False, False],
       [False, False, False],
       [ True, False, False]])

In [152]:
# condition 1 - number of free seats in a row must be >=2
plane.sum(axis=1)

array([3, 1, 0, 1])

In [153]:
plane.sum(axis=1)>=2

array([ True, False, False, False])

In [154]:
# condition 2 - middle seat must be empty
plane[:,1]==True

array([ True, False, False, False])

In [155]:
# success = both condition as True
success = (plane.sum(axis=1)>=2) & (plane[:,1]==True)
success

array([ True, False, False, False])

In [156]:
# just want one row (to continue fight)
success = any(success)
success

True

### Q1

 * Implement function `simulate` to simulate the seating of the passengers and the couple, returning success/failure.
 * Implement function `run_trials` to run the simulation multiple times and calculate the probability success.
 * Then build a table of computed probabilities for increasing number of trials to check for convergence.

In [157]:
def simulate(debug=False):
    seats = np.ones(12, dtype=bool)
    passengers = rng.choice(12, size=7, replace=False)
    seats[passengers] = False
    plane = seats.reshape((4,3))
    success = (plane.sum(axis=1)>=2) & (plane[:,1]==True)

    if debug:
        print("plane:")
        for row,s in zip(plane,success):
            print("\t", row,s)
    return any(success)

simulate(debug=True)


plane:
	 [ True False False] False
	 [False False  True] False
	 [False  True False] False
	 [False  True  True] True


True

In [158]:
def run_trials(n, debug=False):
    trials = [simulate(debug=debug) for _ in range(n)]
    if debug:
        print(trials)
    return sum(trials) / n

run_trials(2, debug=True)

plane:
	 [ True False False] False
	 [ True  True False] True
	 [False False False] False
	 [False  True  True] True
plane:
	 [False  True  True] True
	 [ True False False] False
	 [False  True False] False
	 [False  True False] False
[True, True]


1.0

In [159]:
print(f"{'n':>10s} | {'Pr(avoid fight)':8}\n" + "-"*28)
for k in range(1,6):
    print(f"{10**k:10,} | {run_trials(10**k)}")

         n | Pr(avoid fight)
----------------------------
        10 | 0.6
       100 | 0.79
     1,000 | 0.831
    10,000 | 0.8137
   100,000 | 0.81847


### Q2 

This is an extension of **Q1** so reimplement the functions `simulate` and `run_trials` to include the number of friends.

In [160]:
def simulate(friends=0, debug=False):
    assert friends >= 0 and friends <= 12-7-2, "cannot have negative friends or over book the plane."
    seats = np.ones(12, dtype=bool)
    passengers = rng.choice(12, size=7+friends, replace=False)
    seats[passengers] = False
    plane = seats.reshape((4,3))
    success = (plane.sum(axis=1)>=2) & (plane[:,1]==True)
    
    if debug:
        print("plane:")
        for row,s in zip(plane,success):
            print("\t", row,s)
    return any(success)

simulate(2, debug=True)

plane:
	 [False False False] False
	 [False  True False] False
	 [False False  True] False
	 [ True False False] False


False

In [161]:
def run_trials(friends, n, debug=False):
    trials = [simulate(friends, debug=debug) for _ in range(n)]
    if debug:
        print(trials)
    return sum(trials) / n

run_trials(3, 2, debug=True)

plane:
	 [False False False] False
	 [False False False] False
	 [ True False False] False
	 [ True False False] False
plane:
	 [False False False] False
	 [ True False False] False
	 [False False False] False
	 [ True False False] False
[False, False]


0.0

In [162]:
print(f"{'friends':>10s} | {'Pr(avoid fight)':8}\n" + "-"*26)
for friends in range(0,6):
    n = 10**4 # enough trials to get a good estimate
    pr = run_trials(friends, n)
    print(f"{friends:10} | {pr}")
    if pr<0.25: break

print(f"Minimum number of extra tickets for friends (so that prob of avoiding fight is <25%) is {friends}.")

   friends | Pr(avoid fight)
--------------------------
         0 | 0.8119
         1 | 0.608
         2 | 0.3402
         3 | 0.1244
Minimum number of extra tickets for friends (so that prob of avoiding fight is <25%) is 3.
